<a href="https://colab.research.google.com/github/castrokelly/Data-Science/blob/main/Abordagem_Ensemble_para_Detec%C3%A7%C3%A3o_e_Classifica%C3%A7%C3%A3o_de_Anomalias_em_Po%C3%A7os_de_Petr%C3%B3leo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Abordagem Ensemble para Detecção e Classificação de Anomalias em Poços de Petróleo: um Estudo Aplicado ao Dataset 3W

**Centro de Ciências Matemáticas Aplicadas à Indústria (CeMEAI)**</br>
**Instituto de Ciências Matemáticas e de Computação (ICMC)**</br>
**Universidade de São Paulo**</br>
</br>
Aluna: **Kelly Christine Alvarenga de Castro**
Área de concentração: Ciências de Dados
Orientador: **Prof. Dr. Cláudio Fabiano Motta Toledo**
</br>
---

Este Notebook apresenta o desenvolvimento de um pipeline de aprendizado de máquina para detectar e classificar anomalias em poços de petróleo utilizando o dataset 3W.

A abordagem utiliza um modelo ensemble que, em primeiro lugar, decide se o estado é anômalo (classificação binária: normal vs. anômalo) e, caso seja anômalo, classifica o tipo de evento (rótulos de 1 a 9). O treinamento é realizado com dados de um poço e a validação das métricas é efetuada com dados de outro poço.



Clonagem do repositório contendo o dataset 3W e instalação das bibliotecas necessárias:

In [60]:
!git clone https://github.com/petrobras/3W.git

fatal: destination path '3W' already exists and is not an empty directory.


In [61]:
!pip install scikeras #para a integração do Keras com o scikit-learn
!pip install PyWavelets #para a transformação wavelet

Importação das bibliotecas necessárias para manipulação dos dados, pré-processamento, construção e avaliação dos modelos:

In [62]:
# Importação das Bibliotecas Necessárias
import os
import glob
import numpy as np
import pandas as pd
import pyarrow.parquet as pq
import matplotlib.pyplot as plt
import pywt

from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, StackingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score, classification_report

# Para o modelo LSTM
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from scikeras.wrappers import KerasClassifier

import matplotlib.pyplot as plt
%matplotlib inline


### Funções para Carregamento e Pré-processamento dos Dados

Carregamento dos Dados

Cada arquivo Parquet é carregado e uma coluna extra “instance” (identificando o arquivo) é adicionada para posterior agregação.

In [63]:
def load_well_data(paths):
    dfs = []
    for file in paths:
        df = pd.read_parquet(file)
        df['instance'] = os.path.basename(file)
        dfs.append(df)
    return pd.concat(dfs, ignore_index=True)


Definindo os caminhos dos arquivos

In [64]:
paths_well41 = [
    "/content/3W/dataset/9/WELL-00041_20181013160201.parquet",
    "/content/3W/dataset/9/WELL-00041_20190814154301.parquet",
    "/content/3W/dataset/9/WELL-00041_20190817173916.parquet",
    "/content/3W/dataset/9/WELL-00041_20190819163753.parquet",
    "/content/3W/dataset/9/WELL-00041_20190822143519.parquet",
    "/content/3W/dataset/9/WELL-00041_20190906051807.parquet",
    "/content/3W/dataset/9/WELL-00041_20190917161232.parquet",
    "/content/3W/dataset/9/WELL-00041_20190924202109.parquet",
    "/content/3W/dataset/9/WELL-00041_20190925111412.parquet"
]

paths_well42 = [
    "/content/3W/dataset/9/WELL-00042_20141217142745.parquet",
    "/content/3W/dataset/9/WELL-00042_20141218004109.parquet",
    "/content/3W/dataset/9/WELL-00042_20141218051903.parquet",
    "/content/3W/dataset/9/WELL-00042_20141221190219.parquet",
    "/content/3W/dataset/9/WELL-00042_20141222024535.parquet"
]

Carregando os dados dos poços:

In [65]:
df_well41 = load_well_data(paths_well41)
df_well42 = load_well_data(paths_well42)

## Preparação e Agregação dos Dados:

Nesta etapa:

* Criamos a variável binária `is_anomaly` a partir da coluna “class” (assumindo que 0 = normal e valores 1–9 = anomalia);
* Selecionamos as colunas de sensores (todas as colunas numéricas, exceto as de identificação);
* Agregamos os dados por instância (arquivo) calculando média, desvio padrão, mínimo e máximo de cada variável.
* Essa agregação representa a “definição das melhores features” para o modelo.

In [66]:
# Criando a variável binária para detecção de anomalias
df_well41['is_anomaly'] = df_well41['class'].apply(lambda x: 0 if x == 0 else 1)
df_well42['is_anomaly'] = df_well42['class'].apply(lambda x: 0 if x == 0 else 1)


In [67]:
cols_exclude = ['timestamp', 'instance', 'class', 'is_anomaly']
sensor_cols = [col for col in df_well41.columns if col not in cols_exclude and pd.api.types.is_numeric_dtype(df_well41[col])]


In [68]:
def aggregate_features(df, sensor_cols):
    # Calcula as estatísticas: média, desvio padrão, mínimo e máximo para cada sensor, agrupado por instância
    agg_funcs = ['mean', 'std', 'min', 'max']
    agg_df = df.groupby('instance')[sensor_cols].agg(agg_funcs)
    agg_df.columns = ['_'.join(col).strip() for col in agg_df.columns.values]

    # Recupera os rótulos (assumindo que o valor seja único por instância)
    labels = df.groupby('instance')['class'].first()
    is_anomaly = df.groupby('instance')['is_anomaly'].first()
    agg_df['class'] = labels
    agg_df['is_anomaly'] = is_anomaly
    return agg_df

In [69]:
# Agregando os dados de cada poço
agg_well41 = aggregate_features(df_well41, sensor_cols)
agg_well42 = aggregate_features(df_well42, sensor_cols)

In [70]:
# Visualizando as primeiras linhas do conjunto agregado (poço 41)
agg_well41.head()

,ABER-CKGL_mean,ABER-CKGL_std,ABER-CKGL_min,ABER-CKGL_max,ABER-CKP_mean,ABER-CKP_std,ABER-CKP_min,ABER-CKP_max,ESTADO-DHSV_mean,ESTADO-DHSV_std,...,T-TPT_mean,T-TPT_std,T-TPT_min,T-TPT_max,state_mean,state_std,state_min,state_max,class,is_anomaly
instance,,,,,,,,,,,,,,,,,,,,,
WELL-00041_20181013160201.parquet,38.0,0.0,38.0,38.0,44.00000,0.000000,44.0,44.0,NaN,NaN,...,69.788760,0.000000,69.78876,69.78876,0.0,0.0,0,0,0,1
WELL-00041_20190814154301.parquet,45.0,0.0,45.0,45.0,34.47627,10.226568,3.0,41.0,NaN,NaN,...,69.031564,2.149675,52.14698,70.05357,0.0,0.0,0,0,0,1
WELL-00041_20190817173916.parquet,45.0,0.0,45.0,45.0,41.00000,0.000000,41.0,41.0,NaN,NaN,...,70.020756,0.044872,69.89025,70.15198,0.0,0.0,0,0,0,1
WELL-00041_20190819163753.parquet,45.0,0.0,45.0,45.0,41.00000,0.000000,41.0,41.0,NaN,NaN,...,70.036766,0.037137,69.91747,70.18568,0.0,0.0,0,0,0,1
WELL-00041_20190822143519.parquet,45.0,0.0,45.0,45.0,41.00000,0.000000,41.0,41.0,NaN,NaN,...,70.068498,0.199042,69.76263,70.26575,0.0,0.0,0,0,0,1


In [71]:
print(agg_well41.isna().sum())

ABER-CKGL_mean    0
ABER-CKGL_std     0
ABER-CKGL_min     0
ABER-CKGL_max     0
ABER-CKP_mean     0
                 ..
state_std         0
state_min         0
state_max         0
class             0
is_anomaly        0
Length: 114, dtype: int64


In [72]:
agg_well41_clean = agg_well41.dropna(axis=1)
agg_well42_clean = agg_well42.dropna(axis=1)

In [73]:
# Selecionar apenas as colunas de features (excluindo 'class' e 'is_anomaly')
features_well41 = agg_well41_clean.drop(['class', 'is_anomaly'], axis=1)
features_well42 = agg_well42_clean.drop(['class', 'is_anomaly'], axis=1)

## Pré-Processamento: Normalização e Redução de Dimensionalidade

Agora, vamos separar as features e os rótulos (usando o target binário `is_anomaly`), em seguida:

* Divisão do conjunto do poço 41 em treino e validação;
* Normalização com `StandardScaler`;
* Redução de dimensionalidade com PCA (mantendo 95% da variância).

In [74]:
# Separando features e rótulos para o poço 41
X_train_full = agg_well41.drop(['class','is_anomaly'], axis=1)
y_train_full = agg_well41['is_anomaly']

In [75]:
# Para validação externa (poço 42)
X_test_deploy = agg_well42.drop(['class','is_anomaly'], axis=1)
y_test_deploy = agg_well42['is_anomaly']

In [76]:
# Dividindo os dados do poço 41 em treino e validação (70%/30%)
X_train, X_val, y_train, y_val = train_test_split(X_train_full, y_train_full, test_size=0.3, random_state=42, stratify=y_train_full)


In [77]:
# Normalização
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val)
X_train_full_scaled = scaler.transform(X_train_full)
X_test_deploy_scaled = scaler.transform(X_test_deploy)

/usr/local/lib/python3.11/dist-packages/sklearn/utils/extmath.py:1101: RuntimeWarning: invalid value encountered in divide
  updated_mean = (last_sum + new_sum) / updated_sample_count
/usr/local/lib/python3.11/dist-packages/sklearn/utils/extmath.py:1106: RuntimeWarning: invalid value encountered in divide
  T = new_sum / new_sample_count
/usr/local/lib/python3.11/dist-packages/sklearn/utils/extmath.py:1126: RuntimeWarning: invalid value encountered in divide
  new_unnormalized_variance -= correction**2 / new_sample_count
